In [1]:
# pip install deap
# Iremos criar uma população de vetores preenchidos com 0's e 1's. Deixaremos a população evoluir até conter somente 1's. 

import random
from deap import creator
from deap import base
from deap import tools

# O pacote creator serve para criar uma classe herdando características de alguma outra classe pré-definida. 
# O pacote base será usado para registrar funções dentro de classes
# O pacote tools possui várias funções prontas úteis, como repetir um vetor, fazer mutação, fazer crossover, etc.

In [2]:
# A biblioteca DEAP permite que criemos classes e funções com pouco código, além de já possuir muitas funções prontas. 
# Quando estivermos escolhendo funções e parâmetros para criar nosso algoritmo genético, estaremos criando uma grande classe,
# portanto podemos coletar atributos dessa classe e manipular como quisermos. 

In [3]:
# No pacote creator, além de dar um nome e definir a classe base que servirá como pai, também é possível informar quaisquer 
# elementos extras para essa classe, como variáveis ou funções. 

creator.create(name="ClasseFitness", base=base.Fitness, weights=(1.0,)) # Em weights nós passamos uma tupla de valores
# informando se queremos maximizar (peso positivo) ou minimizar (peso negativo) nossa Fitness Function. 
# Nesse caso, como iremos maximizar um simples objetivo, usamos apenas (1.0,) que é um caso especial de objetivo único. Para
# minimizar o primeiro objetivo e maximizar um segundo objetivo, poderíamos utilizar  weights=(-1.0, 1.0).

creator.create(name="ClasseIndividuos", base=list, fitness=creator.ClasseFitness) # Aqui estamos criando nossa classe responsável por criar os
# indivíduos; ela criará uma lista (por isso passamos list como classe pai) e no parâmetro fitness nós passamos a classe criada 
# anteriormente (ClasseFitness). Isso significa que os indivíduos aqui criados utilizarão a classe de Fitness. 

In [4]:
# Iremos usar um método chamado Toolbox que serve para adicionar (register()) ou remover (unregister()) funções dentro da classe.

modulo_toolbox = base.Toolbox() 
# Em base.Toolbox(), nós informamos uma função e 1 ou mais argumentos que serão passados para essa função. Por exemplo, considere
# o código: import random; random.randint(0,1). A função random.randint() espera receber dois argumentos, um limite inferior e 
# superior para sortear números inteiros aleatoriamente dentro desse intervalo. Se dentro de base.Toolbox() informarmos:
# function=random.randint, 0, 1, isso significa que os argumentos 0, 1 serão passados automaticamente para dentro de randint.
# Além de informar uma função e seus argumentos, também daremos um nome a essa função, no primeiro parâmetro.

# Adicionando os indivíduos:
modulo_toolbox.register("binario_aleatorio", random.randint, 0, 1) # define que serão indivíduos randômicos 0 ou 1
modulo_toolbox.register("individuos", tools.initRepeat, creator.ClasseIndividuos, modulo_toolbox.binario_aleatorio, 100) 
# define que estamos criando indivíduos de 100 números binario_aleatorio cada. Em initRepeat nós precisamos informar a função que
# será chamada e quantidade de vezes que será chamada, bem como onde isso tudo será colocado (primeiro parâmetro).
modulo_toolbox.register("populacao", tools.initRepeat, list, modulo_toolbox.individuos, 300) # define que iremos criar 
# uma população de 300 indivíduos. Repare que, utilizando a função tools.initRepeat, estamos repetindo a função modulo_toolbox.individuos
# 300 vezes e colocando em uma lista.

In [5]:
import random
random.randint(0,8)

5

In [7]:
# Nossa função de avaliação será apenas contar quantos 1's há ao todo em cada indivíduo:
def contabiliza_1s(individuos):
    return sum(individuos), # importante colocar a vírgula, pois isso transforma a saída em um iterable (será necessário depois)

In [8]:
# Os operadores genéticos serão definidos abaixo:
modulo_toolbox.register("avaliar", contabiliza_1s) # estabelece qual será a função de avaliação
modulo_toolbox.register("cruzar", tools.cxTwoPoint) # estabelece como será o crossover
# A função cxTwoPoint recebe duas listas (dois indivíduos) como entrada. Alguns valores de ind1 irão para ind2 e vice-versa. 
# Na prática, será sorteado aleatoriamente um intervalo qualquer dentro dos limites [1, size], sendo 'size' o tamanho da menor 
# lista, e os valores contidos nesse intervalo serão todos permutados entre ind1 e ind2. Por exemplo, se ambos os vetores têm 
# size=100, o intervalo sorteado poderia ser [37:59], isso faria a permuta ind1[37:59] = ind2[37:59], ind2[37:59] = ind1[37:59].

modulo_toolbox.register("mutar", tools.mutFlipBit, indpb=0.05) # estabelece como será a mutação
# A função mutFlipBit transforma o que é 0 em 1 e o que é 1 em 0 (na real, qualquer coisa que não for 1 ela transformará em 0
# também). Mas a função não vai fazer isso em todos os elementos do indivíduo, será apenas naqueles que forem sorteados, sendo
# que a probabilidade de cada valor ser sorteado é informada no parâmetro indpb.

modulo_toolbox.register("selecionar", tools.selTournament, tournsize=3) # estabelece como será a seleção natural
# A função selTournament recebe três parâmetros: os indivíduos, k e tournsize. k é a quantidade de torneios que serão feitos, e
# tournsize é o total de indivíduos que participarão de cada torneio. O indivíduo com maior score em um torneio será o único 
# sobrevivente. Por exemplo: se há 300 indivíduos, k=10 e tornsize=3, isso significa que 3 indivíduos serão sorteados aleatoriamente
# e apenas o indivíduo com maior score entre eles será escolhido. Isso será feito 10 vezes. Nesse caso, seriam retornados 10 indivíduos.
# A cada torneio, o campeão é selecionado e o torneio recomeça com todos. Isso significa que um indivíduo pode ser selecionado
# na lista de sobreviventes/campeões mais de uma vez. 

In [9]:
import statistics

populacao_atual = modulo_toolbox.populacao() # instanciando a população de 300 indivíduos
# A variável populacao_atual consiste em uma lista de 300 listas com 100 elementos em cada.

escores = list(map(modulo_toolbox.avaliar, populacao_atual)) # avaliando o fitness score de cada indivíduo da população. 
# Aqui, a saída de contabiliza_1s precisa ser iterable para a aplicação da função map.

# O objeto modulo_toolbox.avaliar é a função contabiliza_1s() que criamos.
# O objeto escores é uma lista de 300 tuplas.

In [10]:
escores

[(47,),
 (51,),
 (62,),
 (55,),
 (50,),
 (55,),
 (49,),
 (50,),
 (47,),
 (44,),
 (50,),
 (49,),
 (60,),
 (55,),
 (50,),
 (60,),
 (48,),
 (45,),
 (48,),
 (54,),
 (43,),
 (48,),
 (55,),
 (48,),
 (50,),
 (46,),
 (45,),
 (49,),
 (51,),
 (53,),
 (43,),
 (52,),
 (49,),
 (52,),
 (43,),
 (58,),
 (47,),
 (48,),
 (53,),
 (48,),
 (43,),
 (44,),
 (62,),
 (53,),
 (52,),
 (56,),
 (53,),
 (52,),
 (51,),
 (49,),
 (51,),
 (44,),
 (54,),
 (57,),
 (56,),
 (44,),
 (48,),
 (52,),
 (52,),
 (44,),
 (55,),
 (50,),
 (55,),
 (57,),
 (54,),
 (51,),
 (54,),
 (56,),
 (52,),
 (48,),
 (49,),
 (62,),
 (49,),
 (48,),
 (50,),
 (51,),
 (49,),
 (52,),
 (40,),
 (59,),
 (55,),
 (56,),
 (51,),
 (47,),
 (46,),
 (51,),
 (50,),
 (52,),
 (53,),
 (58,),
 (52,),
 (48,),
 (55,),
 (45,),
 (47,),
 (52,),
 (51,),
 (55,),
 (45,),
 (51,),
 (50,),
 (47,),
 (52,),
 (45,),
 (50,),
 (53,),
 (49,),
 (54,),
 (50,),
 (45,),
 (58,),
 (56,),
 (54,),
 (48,),
 (43,),
 (53,),
 (58,),
 (54,),
 (53,),
 (50,),
 (54,),
 (54,),
 (48,),
 (48,),
 (48,),


In [11]:
len(populacao_atual)

300

In [12]:
# Lembre-se que a função população() criou indivíduos, que por sua vez foram criados na ClasseIndividuos, que possui o parâmetro
# fitness. 
populacao_atual[0].fitness

deap.creator.ClasseFitness(())

In [13]:
# Já temos uma lista de indivíduos e uma lista escores. Iremos associar um ao outro no parâmentro fitness que está pronto para
# receber essa informação.
for individuo, score in zip(populacao_atual, escores): # para cada indivíduo da população, iremos atribuir seu score
    individuo.fitness.values = score

# Criando uma lista com todos os fitness values de cada indivíduo da população:
lista_scores = [individuo.fitness.values[0] for individuo in populacao_atual] 

# Definindo algumas constantes (probabilidade de cruzar dois indivíduos e probabilidade de mutar um indivíduo):
prob_cruza, prob_muta = 0.5, 0.2 

In [14]:
# Definindo o código principal que irá rodar todo o programa:

# Iremos definir o critério de parada do nosso programa. Esse critério será pelo menos um indivíduo possuir todos os 
# valores de seu vetor valendo 1 (100 valores 1) ou produzir 1.000 gerações ao todo. O que acontecer primeiro.

# Contador de gerações:
contador_geracoes = 0

# Estabelecendo os critérios de parada e iniciando o loop:
while max(lista_scores) < 100 and contador_geracoes < 1000: 
    contador_geracoes += 1
    print("----GERAÇÃO %i----" % contador_geracoes)

    # Fazendo a seleção natural da próxima geração de indivíduos:
    selecao_sobreviventes = modulo_toolbox.selecionar(populacao_atual, len(populacao_atual)) # retorna os indivíduos de populacao_atual
    sobreviventes = list(map(modulo_toolbox.clone, selecao_sobreviventes)) # salvando esse indivíduos e armazenando numa lista

    # Aplicando crossover e mutação nos descendentes sobreviventes:
    for ind1, ind2 in zip(sobreviventes[::2], sobreviventes[1::2]): # vai pegando pares de 2 em 2 indivíduos até o final
        if random.random() < prob_cruza:
            modulo_toolbox.cruzar(ind1, ind2) # faz a cruza dos dois indivíduos selecionados ao acaso e gera 2 filhos
            # Obs: os 2 filhos gerados são automaticamente colocados nos lugares de seus pais
            del ind1.fitness.values # invalida o fitness do primeiro filho gerado
            del ind2.fitness.values # invalida o fitness do segundo filho gerado

    for mutante in sobreviventes:
        if random.random() < prob_muta:
            modulo_toolbox.mutar(mutante) # aplica mutação no indivíduo selecionado
            del mutante.fitness.values # invalida o fitness desse mutante

    # Calculando o fitness dos novos indivíduos dessa população:
    individuos_sem_fitness = [ind for ind in sobreviventes if not ind.fitness.valid]
    novos_escores = map(modulo_toolbox.avaliar, individuos_sem_fitness)
    for ind, fit in zip(individuos_sem_fitness, novos_escores): # atribuindo (salvando) esses valores em cada indivíduo
        ind.fitness.values = fit

    populacao_atual[:] = sobreviventes # substitui a população que inicia o loop pela nova geração resultante 

    # Em cada iteração, iremos imprimir os resultados do maior e menor fitness score, bem como a média e o desvio padrão
    # dos fitness scores da população atual:

    lista_scores = [ind.fitness.values[0] for ind in populacao_atual] # lista de todos os fitness scores

    media = statistics.mean(lista_scores)
    dvp = statistics.stdev(lista_scores)

    print("Mínimo score: %s" % min(lista_scores))
    print("Máximo: %s" % max(lista_scores))
    print("Média: %s" % media)
    print("Desvio Padrão: %s" % dvp)

----GERAÇÃO 1----
Mínimo score: 43.0
Máximo: 66.0
Média: 54.28333333333333
Desvio Padrão: 4.079033865981211
----GERAÇÃO 2----
Mínimo score: 46.0
Máximo: 69.0
Média: 57.67333333333333
Desvio Padrão: 3.767483953518886
----GERAÇÃO 3----
Mínimo score: 52.0
Máximo: 70.0
Média: 60.846666666666664
Desvio Padrão: 3.1351639615456808
----GERAÇÃO 4----
Mínimo score: 55.0
Máximo: 71.0
Média: 62.95333333333333
Desvio Padrão: 2.910795651818308
----GERAÇÃO 5----
Mínimo score: 54.0
Máximo: 75.0
Média: 64.95
Desvio Padrão: 3.0295368698012157
----GERAÇÃO 6----
Mínimo score: 58.0
Máximo: 78.0
Média: 67.13666666666667
Desvio Padrão: 2.8656637949185955
----GERAÇÃO 7----
Mínimo score: 60.0
Máximo: 77.0
Média: 69.07
Desvio Padrão: 2.9326497886056386
----GERAÇÃO 8----
Mínimo score: 64.0
Máximo: 77.0
Média: 71.02666666666667
Desvio Padrão: 2.6042072126163527
----GERAÇÃO 9----
Mínimo score: 64.0
Máximo: 81.0
Média: 72.64
Desvio Padrão: 2.7230712544581324
----GERAÇÃO 10----
Mínimo score: 67.0
Máximo: 81.0
Média: